## Read data

In [1]:
import pandas as pd

INPUT_FILE_PATH = "../../CGED DATA/CGED-Q_cleaned.csv"
OUTPUT_FILE_PATH = "../../CGED DATA/CGED-Q_new_names_simp.csv"

df = pd.read_csv(INPUT_FILE_PATH, sep="\t")

print(df.shape)
df

c:\Users\sudos\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3526: DtypeWarning: Columns (7,9,11,13,22,24,25,26,27,30,38,39,40,41,42,43,44,45,46,47,50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(3159212, 54)


,year,season,xuhao,interpol,record_number,jicheng_hao,juanhao,cehao,shuming,banben,...,quanxuan_yue,quanxuan_fangshi,juewei,shuoming,assigned_edition,first_in_edition,xuanren_fangshi,banben_nianhao,banben_jijie,person_id
0,1760.75,4.0,7.0,0.0,7.0,1.0,1.0,3.0,滿漢縉紳全書,同陞閣,...,NaN,NaN,NaN,NaN,乾隆25冬.xlsx,0.0,NaN,乾隆,冬,1.760400e+11
1,1760.75,4.0,12.0,0.0,12.0,1.0,1.0,3.0,滿漢縉紳全書,同陞閣,...,NaN,NaN,NaN,NaN,乾隆25冬.xlsx,0.0,NaN,乾隆,冬,1.760400e+11
2,1760.75,4.0,13.0,0.0,13.0,1.0,1.0,3.0,滿漢縉紳全書,同陞閣,...,NaN,NaN,NaN,NaN,乾隆25冬.xlsx,0.0,NaN,乾隆,冬,1.760400e+11
3,1760.75,4.0,25.0,0.0,25.0,1.0,1.0,3.0,滿漢縉紳全書,同陞閣,...,NaN,NaN,NaN,NaN,乾隆25冬.xlsx,0.0,NaN,乾隆,冬,1.760400e+11
4,1760.75,4.0,54.0,0.0,54.0,1.0,1.0,3.0,滿漢縉紳全書,同陞閣,...,NaN,NaN,NaN,NaN,乾隆25冬.xlsx,0.0,NaN,乾隆,冬,1.760400e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3159207,1912.00,1.0,19638.0,0.0,4433594.0,7.0,95.0,1,職官錄,NaN,...,NaN,NaN,NaN,NaN,宣统04春.xlsx,0.0,NaN,宣統,春,1.894315e+11
3159208,1912.00,1.0,19639.0,0.0,4433595.0,7.0,95.0,1,職官錄,NaN,...,NaN,NaN,NaN,NaN,宣统04春.xlsx,0.0,NaN,宣統,春,1.887215e+11
3159209,1912.00,1.0,19641.0,0.0,4433597.0,7.0,95.0,1,職官錄,NaN,...,NaN,NaN,NaN,NaN,宣统04春.xlsx,0.0,NaN,宣統,春,1.903114e+11
3159210,1912.00,1.0,19643.0,0.0,4433599.0,7.0,95.0,1,職官錄,NaN,...,NaN,NaN,NaN,NaN,宣统04春.xlsx,0.0,NaN,宣統,春,1.903100e+11


# Create the simplified name list from Jinshen Lu by using CHAR

In [2]:
from char_converter import CharConverter
converter = CharConverter('v2s')

df["name"] = df["xing"] + df["ming"]

In [3]:
# using CHAR to create a new column "name_simp" for "name"
df["name_simp"] = df["name"].apply(lambda x: converter.convert(x))
df["name_simp"]

0          储麟趾
1           顾镇
2          钱士云
3          僧保住
4          史贻直
          ... 
3159207    陈其杰
3159208    黄桢祥
3159209    葛秉彝
3159210    朱兴汾
3159211    贺廷桂
Name: name_simp, Length: 3159212, dtype: object

In [4]:
jinshenlu_name_simps = df["name_simp"].unique()
print(len(jinshenlu_name_simps))

218025


## Create the simplified name list from CBDB Lu by using CHAR

In [5]:
import sqlite3
conn = sqlite3.connect("latest.db")
c = conn.cursor()
sql = "SELECT c_name_chn FROM BIOG_MAIN WHERE c_dy IN (20)"
c.execute(sql)
rows = c.fetchall()
cbdb_name_simps = [converter.convert(row[0]) for row in rows]
cbdb_name_simps = list(set(cbdb_name_simps))
print(len(cbdb_name_simps))
print(cbdb_name_simps[:10])

105323
['桑成鼎', '安玮', '阎壮图', '溥清', '田志肃', '杨兆鋆', '戴王祥', '赵培之', '叶舟', '郑观吉']


## Find the names from jinshenlu_name_simps that are not in cbdb_name_simps

In [6]:
jinshenlu_name_simps_df = pd.DataFrame(jinshenlu_name_simps, columns=["name_simp"])
cbdb_name_simps_df = pd.DataFrame(cbdb_name_simps, columns=["name_simp"])

jinshenlu_new_names = jinshenlu_name_simps_df[~jinshenlu_name_simps_df["name_simp"].isin(cbdb_name_simps_df["name_simp"])]

print(jinshenlu_new_names.shape)
jinshenlu_new_names[-10:]

(182969, 1)


,name_simp
218015,唐允中
218016,潘鸿观
218017,赵基年
218018,杜俞
218019,李振沄
218020,孙吉焜
218021,赵倜
218022,胡懋芬
218023,杨善德
218024,屈之春


## Save the file

In [7]:
jinshenlu_new_names.to_csv(OUTPUT_FILE_PATH, index=False, sep="\t", encoding="utf-8-sig")